In [ ]:
!pip3 install opencv-python
!pip3 install matplotlib

# apt-get update && apt-get install ffmpeg libsm6 libxext6 -y

In [2]:
import torch
import numpy as np
import argparse
from models.retinanet import build_retinanet
from modules import utils
from torchvision import transforms
import data.transforms as vtf
from data import VideoDataset
import cv2
from PIL import Image
import time


In [3]:
parser = argparse.ArgumentParser(
    description="Training single stage FPN with OHEM, resnet as backbone"
)


In [4]:
# Use CUDA_VISIBLE_DEVICES=0,1,4,6 to select GPUs to use

## Parse arguments
args, _ = utils.get_args(parser)
args = utils.set_args(args)  # set directories and SUBSETS fo datasets
args.MULTI_GPUS = False if args.BATCH_SIZE == 1 else args.MULTI_GPUS

Your working directories are::
LOAD::>  /workspace/ 
SAVE::>  /workspace/road/cache/resnet50I3D512-Pkinetics-b4s8x1x1-roadt3-h3x3x3/
Your model will be initialized using /workspace/kinetics-pt/resnet50I3D.pth


In [5]:
if args.MODE == "train":
    args.TEST_SEQ_LEN = args.SEQ_LEN
else:
    args.SEQ_LEN = args.TEST_SEQ_LEN

args.SEQ_LEN = args.TEST_SEQ_LEN
args.MAX_SEQ_STEP = 1
args.SUBSETS = args.TEST_SUBSETS
full_test = True  # args.MODE != 'train'
args.skip_beggning = 0
args.skip_ending = 0
if args.MODEL_TYPE == "I3D" or "SlowFast":
    args.skip_beggning = 2
    args.skip_ending = 2
elif args.MODEL_TYPE != "C2D":
    args.skip_beggning = 2

skip_step = args.SEQ_LEN - args.skip_beggning

val_transform = transforms.Compose(
    [
        vtf.ResizeClip(args.MIN_SIZE, args.MAX_SIZE),
        vtf.ToTensorStack(),
        vtf.Normalize(mean=args.MEANS, std=args.STDS),
    ]
)

val_dataset = VideoDataset(
    args,
    train=False,
    transform=val_transform,
    skip_step=skip_step,
    full_test=full_test,
)

args.num_classes = val_dataset.num_classes
# one for objectness
args.label_types = val_dataset.label_types
args.num_label_types = val_dataset.num_label_types
args.all_classes = val_dataset.all_classes
args.num_classes_list = val_dataset.num_classes_list
args.num_ego_classes = val_dataset.num_ego_classes
args.ego_classes = val_dataset.ego_classes
args.head_size = 256





database.keys() dict_keys(['2014-06-25-16-45-34_stereo_centre_02', '2014-06-26-09-53-12_stereo_centre_02', '2014-07-14-14-49-50_stereo_centre_01', '2014-07-14-15-42-55_stereo_centre_03', '2014-08-08-13-15-11_stereo_centre_01', '2014-08-11-10-59-18_stereo_centre_02', '2014-11-14-16-34-33_stereo_centre_06', '2014-11-18-13-20-12_stereo_centre_05', '2014-11-21-16-07-03_stereo_centre_01', '2014-11-25-09-18-32_stereo_centre_04', '2014-12-09-13-21-02_stereo_centre_01', '2015-02-03-08-45-10_stereo_centre_02', '2015-02-03-19-43-11_stereo_centre_04', '2015-02-06-13-57-16_stereo_centre_02', '2015-02-13-09-16-26_stereo_centre_02', '2015-02-13-09-16-26_stereo_centre_05', '2015-02-24-12-32-19_stereo_centre_04', '2015-03-03-11-31-36_stereo_centre_01'])






In [6]:
args.ARCH="resnet50"

In [7]:
net = build_retinanet(args).cuda()
if args.MULTI_GPUS:
    # logger.info("\nLets do dataparallel\n")
    net = torch.nn.DataParallel(net)

In [8]:
torch.cuda.is_available()

True

In [9]:
for epoch in args.EVAL_EPOCHS:
    args.MODEL_PATH = args.SAVE_ROOT + "model_{:06d}.pth".format(epoch)
    net.eval()
    net.load_state_dict(torch.load(args.MODEL_PATH))
    net.eval()  # switch net to evaluation mode


/tmp/ipykernel_11857/3472298800.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(args.MODEL_PATH))


In [9]:
import os
base_path = "/workspace/road/rgb-images"
event = "2014-12-09-13-21-02_stereo_centre_01"
im_list = sorted(os.listdir(os.path.join(base_path, event)))
batch_size = 8
batches = [im_list[i:i+batch_size] for i in range(0, len(im_list), batch_size)]
# imgs_paths = [os.path.join(base_path, event, im) for im in im_list]

In [10]:
batches

[['00000.jpg',
  '00001.jpg',
  '00002.jpg',
  '00003.jpg',
  '00004.jpg',
  '00005.jpg',
  '00006.jpg',
  '00007.jpg'],
 ['00008.jpg',
  '00009.jpg',
  '00010.jpg',
  '00011.jpg',
  '00012.jpg',
  '00013.jpg',
  '00014.jpg',
  '00015.jpg'],
 ['00016.jpg',
  '00017.jpg',
  '00018.jpg',
  '00019.jpg',
  '00020.jpg',
  '00021.jpg',
  '00022.jpg',
  '00023.jpg'],
 ['00024.jpg',
  '00025.jpg',
  '00026.jpg',
  '00027.jpg',
  '00028.jpg',
  '00029.jpg',
  '00030.jpg',
  '00031.jpg'],
 ['00032.jpg',
  '00033.jpg',
  '00034.jpg',
  '00035.jpg',
  '00036.jpg',
  '00037.jpg',
  '00038.jpg',
  '00039.jpg'],
 ['00040.jpg',
  '00041.jpg',
  '00042.jpg',
  '00043.jpg',
  '00044.jpg',
  '00045.jpg',
  '00046.jpg',
  '00047.jpg'],
 ['00048.jpg',
  '00049.jpg',
  '00050.jpg',
  '00051.jpg',
  '00052.jpg',
  '00053.jpg',
  '00054.jpg',
  '00055.jpg'],
 ['00056.jpg',
  '00057.jpg',
  '00058.jpg',
  '00059.jpg',
  '00060.jpg',
  '00061.jpg',
  '00062.jpg',
  '00063.jpg'],
 ['00064.jpg',
  '00065.jpg',
  

In [11]:
## For 8 frames only
# import os
# base_path = "/workspace/road/rgb-images"
# event = "2014-12-09-13-21-02_stereo_centre_01"
# im_list = ["00010.jpg", "00011.jpg", "00012.jpg", "00013.jpg", "00014.jpg", "00015.jpg", "00016.jpg", "00017.jpg"]
# imgs_paths = [os.path.join(base_path, event, im) for im in im_list]
# print(imgs_paths)

In [12]:
# import cv2
# images = [cv2.resize(cv2.imread(img_path).astype(np.float32), (512, 704)) for img_path in imgs_paths]
# # images = [cv2.imread(img_path).astype('float32') for img_path in imgs_paths]
# images = torch.tensor([images]).permute(0, 4, 1, 2, 3).cuda(0, non_blocking=True)

In [13]:
# from PIL import Image
# images = [Image.open(img_path).convert("RGB") for img_path in imgs_paths]

In [14]:
# inp_x, inp_y = images[0].size
# print(inp_x, inp_y )

In [15]:
# images = val_transform(images)

In [16]:
# print(images.shape)
# images = images.cuda(0, non_blocking=True)[None, :]
# print(images.shape)


In [17]:
# # torch.cuda.synchronize()
# # Create a tensor with the given size
# images = torch.randn(1, 3, 8, 512, 704).cuda(0, non_blocking=True)

# # Use permute to change the dimensions
# tensor_permuted = images.permute(0, 2, 1, 3, 4)

# # Verify the shape of the resulting tensor
# print(images.shape)


In [18]:
# torch.cuda.synchronize()
# activation = torch.nn.Sigmoid().cuda()

In [19]:
# decoded_boxes, confidences, ego_preds = net(images)
# confidence = activation(confidences)
# seq_len = ego_preds.shape[1]

In [20]:
# decoded_boxes.shape

In [21]:
# confidences.shape

In [22]:
# confidence.shape

In [23]:
inp_x, inp_y = 1920,1080

In [24]:
ratio_factor_x = inp_x/args.MAX_SIZE
ratio_factor_y = inp_y/args.MIN_SIZE

In [ ]:
for batch in batches:
    imgs_paths = [os.path.join(base_path, event, im) for im in batch]
    print(imgs_paths)
    images = val_transform([Image.open(img_path).convert("RGB") for img_path in imgs_paths]).cuda(0, non_blocking=True)[None, :]
    torch.cuda.synchronize()
    activation = torch.nn.Sigmoid().cuda()
    
    decoded_boxes, confidences, ego_preds = net(images)
    confidence = activation(confidences)
    
    for frame_number in range(len(imgs_paths)):
        decoded_boxes_batch = decoded_boxes[0,frame_number]
        confidence_batch = confidence[0, frame_number]
        scores = confidence_batch[:, 0].squeeze().clone()

        _, save_data = utils.filter_detections_for_dumping(
            args, scores, decoded_boxes_batch, confidence_batch)

        del scores, confidence_batch, decoded_boxes_batch
        frame = cv2.imread(imgs_paths[frame_number])
        for det in save_data:
            bbox = det[:4]  # x1, y1, x2, y2
            score = det[4]
            if score>=0.5:
                x1, y1, x2, y2 = map(int, bbox)
                x1, y1, x2, y2 = int(x1*ratio_factor_x), int(y1*ratio_factor_x), int(x2*ratio_factor_x), int(y2*ratio_factor_x)            
                COLORS = (np.random.randint(0,255), np.random.randint(0,255), np.random.randint(0,255))
                cv2.rectangle(frame, (x1, y1), (x2, y2), color=COLORS, thickness=2)  # random unique list(COLORS)
        
        frame_name = "/workspace/testme_"+str(frame_number)+".jpg"
        # cv2.imwrite(frame_name, frame)
    del images, decoded_boxes, confidences, ego_preds, confidence, imgs_paths
    torch.cuda.empty_cache()
    time.sleep(10)



['/workspace/road/rgb-images/2014-12-09-13-21-02_stereo_centre_01/00000.jpg', '/workspace/road/rgb-images/2014-12-09-13-21-02_stereo_centre_01/00001.jpg', '/workspace/road/rgb-images/2014-12-09-13-21-02_stereo_centre_01/00002.jpg', '/workspace/road/rgb-images/2014-12-09-13-21-02_stereo_centre_01/00003.jpg', '/workspace/road/rgb-images/2014-12-09-13-21-02_stereo_centre_01/00004.jpg', '/workspace/road/rgb-images/2014-12-09-13-21-02_stereo_centre_01/00005.jpg', '/workspace/road/rgb-images/2014-12-09-13-21-02_stereo_centre_01/00006.jpg', '/workspace/road/rgb-images/2014-12-09-13-21-02_stereo_centre_01/00007.jpg']


/usr/local/lib/python3.8/dist-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


['/workspace/road/rgb-images/2014-12-09-13-21-02_stereo_centre_01/00008.jpg', '/workspace/road/rgb-images/2014-12-09-13-21-02_stereo_centre_01/00009.jpg', '/workspace/road/rgb-images/2014-12-09-13-21-02_stereo_centre_01/00010.jpg', '/workspace/road/rgb-images/2014-12-09-13-21-02_stereo_centre_01/00011.jpg', '/workspace/road/rgb-images/2014-12-09-13-21-02_stereo_centre_01/00012.jpg', '/workspace/road/rgb-images/2014-12-09-13-21-02_stereo_centre_01/00013.jpg', '/workspace/road/rgb-images/2014-12-09-13-21-02_stereo_centre_01/00014.jpg', '/workspace/road/rgb-images/2014-12-09-13-21-02_stereo_centre_01/00015.jpg']


OutOfMemoryError: CUDA out of memory. Tried to allocate 170.00 MiB. GPU 0 has a total capacity of 5.70 GiB of which 119.19 MiB is free. Process 285425 has 5.57 GiB memory in use. Of the allocated memory 5.30 GiB is allocated by PyTorch, and 146.80 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [24]:
args.GEN_CONF_THRESH=0.3

In [25]:
del net, images, decoded_boxes, confidences, ego_preds, confidence

In [ ]:
decoded_boxes, confidences, ego_preds = net(images)
confidence = activation(confidences)
seq_len = ego_preds.shape[1]

for frame_number in range(len(imgs_paths)):
    decoded_boxes_batch = decoded_boxes[0,frame_number]
    confidence_batch = confidence[0, frame_number]
    scores = confidence_batch[:, 0].squeeze().clone()

    cls_dets, save_data = utils.filter_detections_for_dumping(
        args, scores, decoded_boxes_batch, confidence_batch)

    frame = cv2.imread(imgs_paths[frame_number])
    for det in save_data:
        bbox = det[:4]  # x1, y1, x2, y2
        score = det[4]
        if score>=0.5:
            x1, y1, x2, y2 = map(int, bbox)
            print(np.multiply(y1, ratio_factor_y), y1*ratio_factor_y)
            print(np.multiply(y2, ratio_factor_y), y2*ratio_factor_y)
            x1, y1, x2, y2 = int(x1*ratio_factor_x), int(y1*ratio_factor_x), int(x2*ratio_factor_x), int(y2*ratio_factor_x)            
            COLORS = (np.random.randint(0,255), np.random.randint(0,255), np.random.randint(0,255))
            cv2.rectangle(frame, (x1, y1), (x2, y2), color=COLORS, thickness=2)  # random unique list(COLORS)
    
    frame_name = "/workspace/testme_"+str(frame_number)+".jpg"
    cv2.imwrite(frame_name, frame)